In [1]:
!pip install selenium
!pip install pandas
!pip install openpyxl

In [136]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import selenium.webdriver.chrome.service as service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import requests

from time import sleep
import pandas as pd
import os
import re
import json
from pprint import pprint
from pathlib import Path
import random
import re
import datetime
import openpyxl

OS_NAME = 'linux' if 'posix' in os.name else 'windows'
DESKTOP = {'linux': os.path.join(os.path.join(os.path.expanduser('~')), 'Desktop'),
            'windows': os.path.join(os.path.join(os.environ['USERPROFILE']), 'Desktop')}

# Main class
class Browser:
    def __init__(self) -> None:
        self.chrome_path = ".\\GoogleChromePortable\\App\\Chrome-bin\\chrome.exe"
        self.chromedriver_path = ".\\GoogleChromePortable\\App\\Chrome-bin\\chromedriver.exe"
        self.start_url = "https://www.nfce.sefin.ro.gov.br/consultanfce/consulta.jsp?p=11230804082624003686650180001906841365894182|2|1|4|E2589715C9B0D14ECE8B5DD91E2B1DB1B203406F"
        
        print(self.chrome_path)

        self.options = Options()
#         self.options.add_argument('--incognito')
        self.options.add_argument('log-level=3')  # Supress all warnings and ERRORS

        self.serv = service.Service(self.chromedriver_path)
        self.serv.start()
        self.capabilities = {'chrome.binary': self.chrome_path}

        self.driver = webdriver.Remote(self.serv.service_url, self.capabilities, options=self.options)
        pass

    def go(self, url):
        self.driver.get(url)
        
browser = Browser()
browser.go(browser.start_url)

driver = browser.driver

.\GoogleChromePortable\App\Chrome-bin\chrome.exe


In [138]:
# WebDriverWait(driver, 120).until(EC.invisibility_of_element_located((By.ID, 'captcha')))

versao_detalhada = driver.find_element(By.XPATH, '//*[@id="botao_detalhada"]/a')
versao_detalhada.click()

In [139]:
tabela_dados_nfe = None
linhas_nfe = None
tabela_dados_nfe = driver.find_element(By.ID, 'sefin_NFCe')
linhas_nfe = tabela_dados_nfe.find_elements(By.TAG_NAME, 'tr')

In [140]:
data_emissao = driver.find_elements(By.XPATH, '//*[@id="NFe"]/fieldset[1]/table/tbody/tr/td[4]/span')[0].text
total_nf = driver.find_elements(By.XPATH, '//*[@id="NFe"]/fieldset[1]/table/tbody/tr/td[6]/span')[0].text
cnpj_empresa = driver.find_elements(By.XPATH, '//*[@id="NFe"]/fieldset[2]/table/tbody/tr/td[1]/span')[0].text
nome_empresa = driver.find_elements(By.XPATH, '//*[@id="NFe"]/fieldset[2]/table/tbody/tr/td[2]/span')[0].text
numero_nf = driver.find_elements(By.XPATH, '//*[@id="NFe"]/fieldset[1]/table/tbody/tr/td[3]/span')[0].text

In [141]:
print(data_emissao, total_nf, cnpj_empresa, nome_empresa, numero_nf)

03/08/2023 16:07:09-04:00 677,08 04.082.624/0036-86 IRMAOS GONCALVES COMERCIO E INDUSTRIA LTDA 190684


In [142]:
aba_produtos = driver.find_element(By.XPATH, '//*[@id="tab_tipo_aba"]/li[4]/a')
aba_produtos.click()

In [143]:
div_Prod = driver.find_element(By.ID, 'Prod')
tabela_produtos = div_Prod.find_elements(By.TAG_NAME, 'table')

# Percorra cada tabela e altere o atributo 'style'
for tabela in tabela_produtos:
    driver.execute_script("arguments[0].style.display = 'table';", tabela)


In [144]:
# tabela_toggle = tabela_produtos.find_elements(By.CLASS_NAME, 'toggle')

# print(len(tabela_produtos))

# # # tabela_toggle = div_Prod.find_elements(By.CLASS_NAME, 'toggle')

class Produto:
    data_nf = ''
    nr_nf = ''
    cnpj_empresa = ''
    nome_empresa = ''
    descricao = ''
    qtd = 0
    unidade = ''
    valor = 0
    cod_produto = ''
    cod_ncm = ''
    genero = ''
    cfop = ''
    cod_ean = ''
    valor_un_tributavel = 0
    valor_un_comercializacao = 0
    valor_aprox_tributos = 0
    aliquota_icms = 0
    valor_icms = 0

    
lista_produtos = []

for tabela in tabela_produtos:
    class_name = tabela.get_attribute('class')

    if 'toggle' in class_name or 'toggable' in class_name:
        style = tabela.get_attribute('style')
    
        if 'display: none;' in style:
            print('Detalhes fechado, clicando...')
            tabela.click()
        
        if 'toggle' in class_name:
    
            tabela_toggle = tabela.find_elements(By.TAG_NAME, 'td')
    
            ### Cria novo produto        
            novo_produto =  Produto()
            ### dados do NF
            
            novo_produto.data_nf = data_emissao
            novo_produto.nr_nf = numero_nf
            novo_produto.cnpj_empresa = cnpj_empresa
            novo_produto.nome_empresa = nome_empresa            

            print(tabela_toggle[1].text)

            
            novo_produto.descricao = tabela_toggle[1].text
            novo_produto.qtd = tabela_toggle[2].text
            novo_produto.unidade = tabela_toggle[3].text
            novo_produto.valor = tabela_toggle[4].text

            print(f'Carregando dados do produto {novo_produto.descricao}')

        
    
        elif 'toggable' in class_name:
            print('A tabela é toggable.')

            cod_ncm = None
            cfop = None
            
            inside_table = tabela.find_elements(By.TAG_NAME, 'table')

            table0 = inside_table[0].find_elements(By.TAG_NAME, 'td')
            table1 = inside_table[1].find_elements(By.TAG_NAME, 'td')
            
            novo_produto.cod_produto = table0[0].find_element(By.TAG_NAME, 'span').text
            novo_produto.cod_ncm = table0[1].find_element(By.TAG_NAME, 'span').text
            novo_produto.genero = table0[2].find_element(By.TAG_NAME, 'span').text
            novo_produto.cfop = table0[4].find_element(By.TAG_NAME, 'span').text

            novo_produto.cod_ean = table1[4].find_element(By.TAG_NAME, 'span').text

            novo_produto.valor_un_comercializacao = table1[7].find_element(By.TAG_NAME, 'span').text
            novo_produto.valor_un_tributavel = table1[8].find_element(By.TAG_NAME, 'span').text
            
            novo_produto.valor_aprox_tributos = table1[12].find_element(By.TAG_NAME, 'span').text

            novo_produto.aliquota_icms = table1[19].find_element(By.TAG_NAME, 'span').text
            novo_produto.valor_icms = table1[20].find_element(By.TAG_NAME, 'span').text
            
            lista_produtos.append(novo_produto)

ALHO KGR
Carregando dados do produto ALHO KGR
A tabela é toggable.
REPOLHO KGR
Carregando dados do produto REPOLHO KGR
A tabela é toggable.
CENOURA KGR
Carregando dados do produto CENOURA KGR
A tabela é toggable.
TOMATE KGR
Carregando dados do produto TOMATE KGR
A tabela é toggable.
BETERRABA KGR
Carregando dados do produto BETERRABA KGR
A tabela é toggable.
PIMENTAO KGR
Carregando dados do produto PIMENTAO KGR
A tabela é toggable.
BANANA NANICA KGR
Carregando dados do produto BANANA NANICA KGR
A tabela é toggable.
MACA NACIONAL KGR
Carregando dados do produto MACA NACIONAL KGR
A tabela é toggable.
BOLO FORMIGUEIRO IG KGR
Carregando dados do produto BOLO FORMIGUEIRO IG KGR
A tabela é toggable.
MAMAO HAWAI KGR
Carregando dados do produto MAMAO HAWAI KGR
A tabela é toggable.
CEBOLA KGR
Carregando dados do produto CEBOLA KGR
A tabela é toggable.
BATATA MONALISA KGR
Carregando dados do produto BATATA MONALISA KGR
A tabela é toggable.
PEPINO KGR
Carregando dados do produto PEPINO KGR
A tabe

In [145]:
df = pd.DataFrame([vars(p) for p in lista_produtos])
df['qtd'] = df['qtd'].str.replace(',', '.').astype(float)
df['valor'] = df['valor'].str.replace(',', '.').astype(float)
df['valor_un_comercializacao'] = df['valor_un_comercializacao'].str.replace(',', '.').astype(float)
df['valor_un_tributavel'] = df['valor_un_tributavel'].str.replace(',', '.').astype(float)
df['valor_aprox_tributos'] = df['valor_aprox_tributos'].str.replace(',', '.').astype(float)
# df['aliquota_icms'] = df['aliquota_icms'].str.replace(',', '.').astype(float)
# df['valor_icms'] = df['valor_icms'].str.replace(',', '.').astype(float)
df


,data_nf,nr_nf,cnpj_empresa,nome_empresa,descricao,qtd,unidade,valor,cod_produto,cod_ncm,genero,cfop,cod_ean,valor_un_comercializacao,valor_un_tributavel,valor_aprox_tributos,aliquota_icms,valor_icms
0,03/08/2023 16:07:09-04:00,190684,04.082.624/0036-86,IRMAOS GONCALVES COMERCIO E INDUSTRIA LTDA,ALHO KGR,0.314,KG,6.18,00000406,07032090,,5102,SEM GTIN,19.6815,19.6815,2.28,"12,00","0,74"
1,03/08/2023 16:07:09-04:00,190684,04.082.624/0036-86,IRMAOS GONCALVES COMERCIO E INDUSTRIA LTDA,REPOLHO KGR,0.894,KG,5.35,00000437,07049000,,5102,SEM GTIN,5.9843,5.9843,1.09,,
2,03/08/2023 16:07:09-04:00,190684,04.082.624/0036-86,IRMAOS GONCALVES COMERCIO E INDUSTRIA LTDA,CENOURA KGR,0.478,KG,2.77,00000512,07061000,,5102,SEM GTIN,5.7950,5.7950,0.57,,
3,03/08/2023 16:07:09-04:00,190684,04.082.624/0036-86,IRMAOS GONCALVES COMERCIO E INDUSTRIA LTDA,TOMATE KGR,0.914,KG,8.21,00000635,07020000,,5102,SEM GTIN,8.9825,8.9825,1.68,,
4,03/08/2023 16:07:09-04:00,190684,04.082.624/0036-86,IRMAOS GONCALVES COMERCIO E INDUSTRIA LTDA,BETERRABA KGR,0.444,KG,1.55,00000666,07069000,,5102,SEM GTIN,3.4910,3.4910,0.32,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,03/08/2023 16:07:09-04:00,190684,04.082.624/0036-86,IRMAOS GONCALVES COMERCIO E INDUSTRIA LTDA,FARINHA MAND TIA ELIZA PREM 490 GR,1.000,PACOTE,5.06,09038998,11062000,,5102,602883822818,5.0600,5.0600,1.29,"12,00","0,61"
80,03/08/2023 16:07:09-04:00,190684,04.082.624/0036-86,IRMAOS GONCALVES COMERCIO E INDUSTRIA LTDA,VANISH REFIL PO WHITE 240 GR,1.000,UNID,9.80,09167964,34025000,,5102,7891035001468,9.8000,9.8000,3.20,"17,50","1,72"
81,03/08/2023 16:07:09-04:00,190684,04.082.624/0036-86,IRMAOS GONCALVES COMERCIO E INDUSTRIA LTDA,MACAR LIANE LASANHA PRE COZ 200 GR,1.000,UNID,4.78,09250840,19021900,,5102,7896080830339,4.7800,4.7800,1.48,"17,50","0,84"
82,03/08/2023 16:07:09-04:00,190684,04.082.624/0036-86,IRMAOS GONCALVES COMERCIO E INDUSTRIA LTDA,FILEZINHO FGO FRIATO IQF 1 KG,1.000,PACOTE,15.99,09489578,02071400,,5405,7896733303616,15.9900,15.9900,4.07,,


In [146]:
df.to_excel(f'cupom-{numero_nf}.xlsx')